In [1]:
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForSeq2SeqLM

# config = PeftConfig.from_pretrained("lorahub/flan_t5_large-super_glue_wic")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
# model = PeftModel.from_pretrained(model, "lorahub/flan_t5_large-super_glue_wic")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


config.json: 100%|██████████| 662/662 [00:00<00:00, 2.77MB/s]
model.safetensors: 100%|██████████| 3.13G/3.13G [00:15<00:00, 200MB/s] 
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 652kB/s]
adapter_model.bin: 100%|██████████| 19.0M/19.0M [00:00<00:00, 36.8MB/s]


In [6]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
# input_ids = tokenizer.encode("""
# Sentence: Other title changes included Lord Steven Regal and The Nasty Boys winning the World Television Championship and the World Tag Team Championship respectively.
# Facts:
# - Lord Steven Regal wan the World Television Championship. 
# - The Nasty Boys wan and the World Tag Team Championship.

# Sentence: The parkway was opened in 2001 after just under a year of construction and almost two decades of community requests.
# Facts:
# """, return_tensors="pt")
# outputs = model.generate(input_ids=input_ids)
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# print(generated_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The parkway was opened in 2001.


In [1]:
import transformers

print(transformers.__version__)

from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

# model_checkpoint = "EleutherAI/gpt-neox-20b"
model_checkpoint =  "distilgpt2"
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)
trainer.train()

4.32.1


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
